# 第9回講義 宿題

## 課題. Theanoを用いてVatiational auto-encoder (VAE) を実装せよ

### 注意

- homework関数を完成させて提出してください
    - **訓練データのtrain_Xのみ**が与えられます
    - train_Xをtrain_Xとvalid_Xに分けるなどしてモデルを学習させてください
    - **test関数を戻り値**としてください (下に書いてあります)
- **test_Xに対するLower Boundで評価**します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください

次のような内容のコードが**事前**に実行されます

```python
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
from theano.tensor.shared_randomstreams import RandomStreams

import numpy as np
import theano
import theano.tensor as T

rng = np.random.RandomState(1234)
theano_rng = RandomStreams(rng.randint(1234))


mnist = fetch_mldata('MNIST original')
mnist_X = shuffle(mnist.data.astype('float32'), random_state=42)

mnist_X = mnist_X / 255.0

train_X, test_X = train_test_split(mnist_X, test_size=0.2, random_state=??)
```

次のセルのhomework関数を完成させて提出してください
- **上記のコード以外で必要なもの**は全て書いてください

In [3]:
def homework(train_X):
    import time
    start_time = time.time()
    
    # WRITE ME
    class Autoencoder:
    # Constructor
        def __init__(self, visible_dim, hidden_dim, W, function):
            self.visible_dim = visible_dim
            self.hidden_dim = hidden_dim
            self.function = function
            self.W = W
            self.a = theano.shared(np.zeros(visible_dim).astype('float32'),
                                   name='a')
            self.b = theano.shared(np.zeros(hidden_dim).astype('float32'),
                                   name='b')
            self.params = [self.W, self.a, self.b]

    # Encoder
        def encode(self, x):
            u = T.dot(x, self.W) + self.b
            y = self.function(u)
            return y

    # Decoder
        def decode(self, x):
            u = T.dot(x, self.W.T) + self.a
            y = self.function(u)
            return y

    # Forward Propagation
        def f_prop(self, x):
            y = self.encode(x)
            reconst_x = self.decode(y)
            return reconst_x

    # Reconstruction Error
        def reconst_error(self, x, noise):
            tilde_x = x*noise
            reconst_x = self.f_prop(tilde_x)
            error = T.mean(T.sum(T.nnet.binary_crossentropy(reconst_x, x), axis=1))
            return error, reconst_x
    
    def Adam(params, g_params, lr=0.0002, b1=0.1, b2=0.001, e=1e-8):
        updates = []
        i = theano.shared(np.float32(0.), name='i')
        i_t = i + 1.
        fix1 = 1. - (1. - b1)**i_t
        fix2 = 1. - (1. - b2)**i_t
        lr_t = lr * (T.sqrt(fix2) / fix1)
        for p, g in zip(params, g_params):
            m = theano.shared(p.get_value() * 0., name='m')
            v = theano.shared(p.get_value() * 0., name='v')
            m_t = (b1 * g) + ((1. - b1) * m)
            v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
            g_t = m_t / (T.sqrt(v_t) + e)
            p_t = p - (lr_t * g_t)
            updates.append((m, m_t))
            updates.append((v, v_t))
            updates.append((p, p_t))
        updates.append((i, i_t))
        return updates

# Define the Layer class 

    class Layer:
    # Constructor
        def __init__(self, in_dim, out_dim, function):
            self.in_dim = in_dim
            self.out_dim = out_dim
            self.function = function
            self.W = theano.shared(np.random.uniform(low=-0.08, high=0.08,
                                               size=(in_dim, out_dim)
                                               ).astype('float32'),
                                   name='W')
            self.b = theano.shared(np.zeros(out_dim).astype('float32'), name='b')
            self.params = [self.W, self.b]
            self.set_pretraining()

    # Forward Propagation
        def f_prop(self, x):
            self.u = T.dot(x, self.W) + self.b
            self.z = self.function(self.u)
            return self.z
    
        def get_mask(self):
            a = np.random.rand(self.out_dim) < self.possibility
            return a*np.float32(1.0)

    # Set Pretraining
        def set_pretraining(self):
            ae = Autoencoder(self.in_dim, self.out_dim, self.W, self.function)

            k = T.fmatrix(name='k')
            noise = T.fmatrix(name='noise')

            cost, reconst_x = ae.reconst_error(k, noise)
            params = ae.params
            g_params = T.grad(cost=cost, wrt=params)
            updates = Adam(params, g_params)

            self.pretraining = theano.function(inputs=[k, noise],
                                                outputs=[cost, reconst_x],
                                                updates=updates,
                                                allow_input_downcast=True,
                                                name='pretraining')

            hidden = ae.encode(k)
            self.encode_function = theano.function(inputs=[k], outputs=hidden,
                                                   allow_input_downcast=True,
                                                   name='encode_function')
    class VAE:
        def __init__(self, q, p, random=1234):
            self.q = q
            self.p = p
            self.srng = RandomStreams(seed=random)

        def q_f_prop(self, x):
        # Gaussian MLP
            params = []
            layer_out = x
            for i, layer in enumerate(self.q[:-2]):
                params += layer.params
                layer_out = layer.f_prop(layer_out)

            params += self.q[-2].params
            mean = self.q[-2].f_prop(layer_out)

            params += self.q[-1].params
            var = self.q[-1].f_prop(layer_out)

            return mean, var, params

        def p_f_prop(self, x):
        # Bernoulli MLP
            params = []
            layer_out = x
            for i, layer in enumerate(self.p):
                params += layer.params
                layer_out = layer.f_prop(layer_out)
            mean = layer_out

            return mean, params

        def lower_bound(self, x):
        # Encode
            mean, var, q_params = self.q_f_prop(x)
            KL = -0.5*T.mean(T.sum(1 + T.log(var) - mean**2 - var, axis=1))

            epsilon = self.srng.normal(mean.shape)
            z = mean + T.sqrt(var)*epsilon

        # Decode
            _x, p_params = self.p_f_prop(z)
            log_likelihood = T.mean(T.sum(x*T.log(_x) + (1 - x)*T.log(1 - _x),
                                          axis=1))

            params = q_params + p_params

            lower_bound = [-KL, log_likelihood]

            return lower_bound, params
            
    z_dim = 10

    # Gaussian MLP
    q = [
        Layer(784, 500, T.nnet.sigmoid),
        Layer(500, 500, T.nnet.sigmoid),
        Layer(500, 300, T.nnet.sigmoid),
        Layer(300, 200, T.nnet.sigmoid),
        Layer(200, z_dim, lambda  x: x),    # mean
        Layer(200, z_dim, T.nnet.softplus)  # variance
    ]

    # Bernoulli MLP
    p = [
        Layer(z_dim, 200, T.nnet.relu),
        Layer(200, 400, T.nnet.relu),
        Layer(400, 400, T.nnet.relu),
        Layer(400, 784, T.nnet.sigmoid)
    ]
    
    
    X = np.copy(train_X)
    
    for l, layer in enumerate(q[:4]):
        corruption_level = np.float32(0.3)
        batch_size = 100
        n_batches = X.shape[0] // batch_size

        for epoch in range(121):
            X = shuffle(X)
            err_all = []
            cost = []
            for i in range(0, n_batches):
                start = i*batch_size
                end = start + batch_size

                noise = np.random.binomial(size=X[start:end].shape, n=1,
                                     p=1-corruption_level)
                err, reconst_X = layer.pretraining(X[start:end], noise)
                err_all.append(err)
            cost.append(np.mean(err))
            #if epoch%40 == 0:
            #    print('Pretraining:: layer:%d, Epoch:%d, Error:%lf ,Time: %d' %
            #      (l, epoch, np.mean(err),(time.time()-start_time)/60))
        

        X = layer.encode_function(X)
    
    model = VAE(q, p)

    x = T.fmatrix('x')
    lower_bound, params = model.lower_bound(x)
    g_params = T.grad(-T.sum(lower_bound), params)
    updates = Adam(params, g_params)

    train = theano.function(inputs=[x], outputs=lower_bound, updates=updates,
                            allow_input_downcast=True, name='train')
    batch_size = 100
    n_batches = train_X.shape[0]//batch_size

    for epoch in range(201):
        rng.shuffle(train_X)
        lowerbound_all = []
        for i in range(n_batches):
            start = i*batch_size
            end = start + batch_size
            lowerbound = train(train_X[start:end])
            lowerbound_all.append(lowerbound)

        lowerbound_all = np.mean(lowerbound_all, axis=0)
        #test = theano.function(inputs=[x], outputs=T.sum(lower_bound),
        #                   allow_input_downcast=True, name='test')
        #test_lowerbound = test(test_X)
        #if epoch%20==0:
        #    print('Epoch:%d, Train Lower Bound:%lf (%lf, %lf), Test Lower Bound:%lf,Time:%d minutes' %
        #          (epoch, np.sum(lowerbound_all), lowerbound_all[0], lowerbound_all[1],
        #           test_lowerbound, (time.time()-start_time)/60))
          

    # 以下の行はそのままでsubmitしてください
    test = theano.function(inputs=[x], outputs=T.sum(lower_bound),
                           allow_input_downcast=True, name='test')
    return test


Pretraining:: layer:0, Epoch:0, Error:142.332230 ,Time: 3
Pretraining:: layer:0, Epoch:40, Error:68.064461 ,Time: 4
Pretraining:: layer:0, Epoch:80, Error:63.981068 ,Time: 6
Pretraining:: layer:0, Epoch:120, Error:63.706905 ,Time: 7
Pretraining:: layer:1, Epoch:0, Error:221.192657 ,Time: 7
Pretraining:: layer:1, Epoch:40, Error:163.834656 ,Time: 8
Pretraining:: layer:1, Epoch:80, Error:171.731674 ,Time: 9
Pretraining:: layer:1, Epoch:120, Error:169.650192 ,Time: 10
Pretraining:: layer:2, Epoch:0, Error:164.524567 ,Time: 10
Pretraining:: layer:2, Epoch:40, Error:104.905350 ,Time: 11
Pretraining:: layer:2, Epoch:80, Error:101.980370 ,Time: 12
Pretraining:: layer:2, Epoch:120, Error:99.877098 ,Time: 13
Pretraining:: layer:3, Epoch:0, Error:144.592484 ,Time: 13
Pretraining:: layer:3, Epoch:40, Error:100.840309 ,Time: 14
Pretraining:: layer:3, Epoch:80, Error:98.162422 ,Time: 15
Pretraining:: layer:3, Epoch:120, Error:94.865196 ,Time: 15
Epoch:0, Train Lower Bound:-221.388809 (-2.930732, -2

In [2]:
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
from theano.tensor.shared_randomstreams import RandomStreams

import numpy as np
import theano
import theano.tensor as T

rng = np.random.RandomState(1234)
theano_rng = RandomStreams(rng.randint(1234))


mnist = fetch_mldata('MNIST original')
mnist_X = shuffle(mnist.data.astype('float32'), random_state=42)

mnist_X = mnist_X / 255.0

train_X, test_X = train_test_split(mnist_X, test_size=0.2, random_state=1)

Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


In [52]:
class Autoencoder:
    # Constructor
    def __init__(self, visible_dim, hidden_dim, W, function):
        self.visible_dim = visible_dim
        self.hidden_dim = hidden_dim
        self.function = function
        self.W = W
        self.a = theano.shared(np.zeros(visible_dim).astype('float32'),
                               name='a')
        self.b = theano.shared(np.zeros(hidden_dim).astype('float32'),
                               name='b')
        self.params = [self.W, self.a, self.b]

    # Encoder
    def encode(self, x):
        u = T.dot(x, self.W) + self.b
        y = self.function(u)
        return y

    # Decoder
    def decode(self, x):
        u = T.dot(x, self.W.T) + self.a
        y = self.function(u)
        return y

    # Forward Propagation
    def f_prop(self, x):
        y = self.encode(x)
        reconst_x = self.decode(y)
        return reconst_x

    # Reconstruction Error
    def reconst_error(self, x, noise):
        tilde_x = x*noise
        reconst_x = self.f_prop(tilde_x)
        error = T.mean(T.sum(T.nnet.binary_crossentropy(reconst_x, x), axis=1))
        return error, reconst_x

In [53]:
class Layer:
    # Constructor
    def __init__(self, in_dim, out_dim, function):
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.function = function
        self.W = theano.shared(rng.uniform(low=-0.08, high=0.08,
                                           size=(in_dim, out_dim)
                                           ).astype('float32'),
                               name='W')
        self.b = theano.shared(np.zeros(out_dim).astype('float32'), name='b')
        self.params = [self.W, self.b]

        self.set_pretraining()

    # Forward Propagation
    def f_prop(self, x):
        self.u = T.dot(x, self.W) + self.b
        self.z = self.function(self.u)
        return self.z

    # Set Pretraining
    def set_pretraining(self):
        ae = Autoencoder(self.in_dim, self.out_dim, self.W, self.function)

        k = T.fmatrix(name='k')
        noise = T.fmatrix(name='noise')

        cost, reconst_k = ae.reconst_error(k, noise)
        params = ae.params
        g_params = T.grad(cost=cost, wrt=params)
        updates = Adam(params,g_params)

        self.pretraining = theano.function(inputs=[k, noise],
                                           outputs=[cost, reconst_k],
                                           updates=updates,
                                           allow_input_downcast=True,
                                           name='pretraining')

        hidden = ae.encode(k)
        self.encode_function = theano.function(inputs=[k], outputs=hidden,
                                               allow_input_downcast=True,
                                               name='encode_function')

In [ ]:
from collections import OrderedDict
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
from theano.tensor.shared_randomstreams import RandomStreams

import numpy as np
import theano
import theano.tensor as T

rng = np.random.RandomState(1234)
theano_rng = RandomStreams(rng.randint(1234))


def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X = shuffle(mnist.data.astype('float32'), random_state=42)

    mnist_X = mnist_X / 255.0

    train_X, test_X = train_test_split(mnist_X, test_size=0.2, random_state=42)

    return (train_X, test_X)


def check_homework():
    train_X, test_X = load_mnist()

    # validate for small dataset
    train_X_mini = train_X[:1000]
    test_X_mini = test_X[:1000]

    test = homework(train_X_mini)
    lower_bound = test(test_X_mini)
    return lower_bound

if 'homework' in globals():
    result = check_homework()

    print("No Error Occured!")

In [54]:
class VAE:
    def __init__(self, q, p, random=1234):
        self.q = q
        self.p = p
        self.srng = RandomStreams(seed=random)

    def q_f_prop(self, x):
        # Gaussian MLP
        params = []
        layer_out = x
        for i, layer in enumerate(self.q[:-2]):
            params += layer.params
            layer_out = layer.f_prop(layer_out)

        params += self.q[-2].params
        mean = self.q[-2].f_prop(layer_out)

        params += self.q[-1].params
        var = self.q[-1].f_prop(layer_out)

        return mean, var, params

    def p_f_prop(self, x):
        # Bernoulli MLP
        params = []
        layer_out = x
        for i, layer in enumerate(self.p):
            params += layer.params
            layer_out = layer.f_prop(layer_out)
        mean = layer_out

        return mean, params

    def lower_bound(self, x):
        # Encode
        mean, var, q_params = self.q_f_prop(x)
        KL = -0.5*T.mean(T.sum(1 + T.log(var) - mean**2 - var, axis=1))

        epsilon = self.srng.normal(mean.shape)
        z = mean + T.sqrt(var)*epsilon

        # Decode
        _x, p_params = self.p_f_prop(z)
        log_likelihood = T.mean(T.sum(x*T.log(_x) + (1 - x)*T.log(1 - _x),
                                      axis=1))

        params = q_params + p_params

        lower_bound = [-KL, log_likelihood]

        return lower_bound, params

In [55]:
def Adam(params, g_params, lr=0.0002, b1=0.1, b2=0.001, e=1e-8):
    updates = []
    i = theano.shared(np.float32(0.), name='i')
    i_t = i + 1.
    fix1 = 1. - (1. - b1)**i_t
    fix2 = 1. - (1. - b2)**i_t
    lr_t = lr * (T.sqrt(fix2) / fix1)
    for p, g in zip(params, g_params):
        m = theano.shared(p.get_value() * 0., name='m')
        v = theano.shared(p.get_value() * 0., name='v')
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
        g_t = m_t / (T.sqrt(v_t) + e)
        p_t = p - (lr_t * g_t)
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    updates.append((i, i_t))
    return updates

In [56]:
z_dim = 10

# Gaussian MLP
q = [
    Layer(784, 500, T.nnet.sigmoid),
    Layer(500, 500, T.nnet.sigmoid),
    Layer(500, 300, T.nnet.sigmoid),
    Layer(300, 200, T.nnet.sigmoid),
    Layer(200, z_dim, lambda  x: x),    # mean
    Layer(200, z_dim, T.nnet.softplus)  # variance
]

# Bernoulli MLP
p = [
    Layer(z_dim, 200, T.nnet.relu),
    Layer(200, 200, T.nnet.relu),
    Layer(200, 784, T.nnet.sigmoid)
]

In [57]:
X = np.copy(train_X)
for l, layer in enumerate(q[0:3]):
    corruption_level = np.float32(0.3)
    batch_size = 100
    n_batches = X.shape[0] // batch_size

    for epoch in range(201):
        X = shuffle(X)
        err_all = []
        for i in range(0, n_batches):
            start = i*batch_size
            end = start + batch_size

            noise = rng.binomial(size=X[start:end].shape, n=1,
                                 p=1-corruption_level)
            err, reconst_X = layer.pretraining(X[start:end], noise)
            err_all.append(err)
        if epoch%40 == 0:
            print('Pretraining:: layer:%d, Epoch:%d, Error:%lf' %
                  (l, epoch, np.mean(err)))
    X = layer.encode_function(X)

Pretraining:: layer:0, Epoch:0, Error:138.966324
Pretraining:: layer:0, Epoch:20, Error:73.357460
Pretraining:: layer:0, Epoch:40, Error:67.467400
Pretraining:: layer:0, Epoch:60, Error:65.774094
Pretraining:: layer:0, Epoch:80, Error:62.726475
Pretraining:: layer:0, Epoch:100, Error:63.868881
Pretraining:: layer:0, Epoch:120, Error:62.721699
Pretraining:: layer:0, Epoch:140, Error:62.058556
Pretraining:: layer:0, Epoch:160, Error:64.605103
Pretraining:: layer:0, Epoch:180, Error:63.222862
Pretraining:: layer:0, Epoch:200, Error:62.131535
Pretraining:: layer:1, Epoch:0, Error:226.017380
Pretraining:: layer:1, Epoch:20, Error:172.983948
Pretraining:: layer:1, Epoch:40, Error:169.961273
Pretraining:: layer:1, Epoch:60, Error:165.851440
Pretraining:: layer:1, Epoch:80, Error:165.529633
Pretraining:: layer:1, Epoch:100, Error:169.721252
Pretraining:: layer:1, Epoch:120, Error:165.525391
Pretraining:: layer:1, Epoch:140, Error:162.754868
Pretraining:: layer:1, Epoch:160, Error:163.850159
Pr

In [59]:
model = VAE(q, p)

x = T.fmatrix('x')
lower_bound, params = model.lower_bound(x)

g_params = T.grad(-T.sum(lower_bound), params)
updates = Adam(params, g_params)

train = theano.function(inputs=[x], outputs=lower_bound, updates=updates,
                        allow_input_downcast=True, name='train')
test = theano.function(inputs=[x], outputs=T.sum(lower_bound),
                       allow_input_downcast=True, name='test')

In [60]:
batch_size = 100
n_batches = train_X.shape[0]//batch_size

for epoch in range(101):
    rng.shuffle(train_X)
    lowerbound_all = []
    for i in range(n_batches):
        start = i*batch_size
        end = start + batch_size
        lowerbound = train(train_X[start:end])
        lowerbound_all.append(lowerbound)

    lowerbound_all = np.mean(lowerbound_all, axis=0)
    test_lowerbound = test(test_X)
    if epoch%20 == 0:
        print('Epoch:%d, Train Lower Bound:%lf (%lf, %lf), Test Lower Bound:%lf' %
              (epoch, np.sum(lowerbound_all), lowerbound_all[0], lowerbound_all[1],
               test_lowerbound))

Epoch:0, Train Lower Bound:-240.235504 (-6.583884, -233.651627), Test Lower Bound:-198.783920
Epoch:20, Train Lower Bound:-113.230934 (-17.231216, -95.999718), Test Lower Bound:-113.339645
Epoch:40, Train Lower Bound:-107.725426 (-18.070480, -89.654945), Test Lower Bound:-108.178543
Epoch:60, Train Lower Bound:-105.155441 (-18.403482, -86.751961), Test Lower Bound:-105.879807
Epoch:80, Train Lower Bound:-103.579178 (-18.626209, -84.952965), Test Lower Bound:-104.566132
Epoch:100, Train Lower Bound:-102.492180 (-18.816610, -83.675568), Test Lower Bound:-103.636269


In [51]:
z_dim = 10

# Gaussian MLP
q = [
    Layer(784, 500, T.nnet.sigmoid),
    Layer(500, 300, T.nnet.sigmoid),
    Layer(300, 200, T.nnet.sigmoid),
    Layer(200, z_dim, lambda  x: x),    # mean
    Layer(200, z_dim, T.nnet.softplus)
]  

X = np.copy(train_X)
for l, layer in enumerate(q[:3]):
    corruption_level = np.float32(0.3)
    batch_size = 100
    n_batches = X.shape[0] // batch_size

    for epoch in range(201):
        X = shuffle(X)
        err_all = []
        for i in range(0, n_batches):
            start = i*batch_size
            end = start + batch_size

            noise = rng.binomial(size=X[start:end].shape, n=1,
                                 p=1-corruption_level)
            err, reconst_X = layer.pretraining(X[start:end], noise)
            err_all.append(err)
        if epoch%20 == 0:
            print('Pretraining:: layer:%d, Epoch:%d, Error:%lf' %
                  (l, epoch, np.mean(err)))
    X = layer.encode_function(X)

Pretraining:: layer:0, Epoch:9, Error:85.137848
Pretraining:: layer:0, Epoch:19, Error:80.135284
Pretraining:: layer:0, Epoch:29, Error:72.581833
Pretraining:: layer:0, Epoch:39, Error:70.498108
Pretraining:: layer:0, Epoch:49, Error:67.304733
Pretraining:: layer:0, Epoch:59, Error:69.861076
Pretraining:: layer:0, Epoch:69, Error:68.732666
Pretraining:: layer:0, Epoch:79, Error:65.270439
Pretraining:: layer:0, Epoch:89, Error:64.035110
Pretraining:: layer:0, Epoch:99, Error:65.899071
Pretraining:: layer:1, Epoch:9, Error:187.158630
Pretraining:: layer:1, Epoch:19, Error:183.526947
Pretraining:: layer:1, Epoch:29, Error:175.565964
Pretraining:: layer:1, Epoch:39, Error:173.379272
Pretraining:: layer:1, Epoch:49, Error:169.216644
Pretraining:: layer:1, Epoch:59, Error:170.476974
Pretraining:: layer:1, Epoch:69, Error:167.229645
Pretraining:: layer:1, Epoch:79, Error:175.302750
Pretraining:: layer:1, Epoch:89, Error:172.085159
Pretraining:: layer:1, Epoch:99, Error:167.947540
Pretraining: